In [2]:
import csv
unique_entities_strict = set()
unique_entities_lenient = set()
entities = []

text = []
labels = []


with open("./output/ner-crf-training-data.tsv", newline = '') as lines:  
    line_reader = csv.reader(lines, delimiter='\t')
    sentence = []
    entities = []
    pre_entity = "NEVER_EXISTS_ENTITY"
    for idx, line in enumerate(line_reader):
        ## . signal the begining of a new line
        ## skip empty line
        if line == []:
            continue
        if line[0] == '.':
            text.append(" ".join(sentence + [line[0]]))
            labels.append(" ".join(entities + [line[1]]))
            sentence, entities = [] , []
            strict_entity = line[1]
        else:
            sentence.append(line[0])
            if line[1] != "O" and pre_entity == line[1]:
                entities.append("I-"+line[1])
                strict_entity = "I-"+line[1]
            elif line[1] != "O" and pre_entity != line[1]:
                entities.append("B-"+line[1])
                strict_entity = "B-"+line[1]
            else:
                entities.append(line[1])
                strict_entity = line[1]
        
        pre_entity = line[1]
        
        
        unique_entities_strict.add(strict_entity)
        unique_entities_lenient.add(line[1])
        entities.append(line[1])


NameError: name 'strict_entity' is not defined

In [ ]:
import pandas as pd
new_df = pd.DataFrame({"text": text, "labels": labels})
new_df.head()
new_df.to_csv("./output/ner.csv")

,text,labels
0,A 48 year - old female presented with vaginal ...,O B-Age I-Age I-Age I-Age B-Sex B-Clinical_eve...
1,Upon diagnosis of invasive non-keratinizing SC...,O O O B-Detailed_description I-Detailed_descri...
2,Pathological examination revealed that the tum...,B-Diagnostic_procedure I-Diagnostic_procedure ...
3,"5 months after this surgery , the woman underw...",B-Date I-Date I-Date I-Date I-Date O O O O B-D...
4,"Two years later , the patient presented with a...",B-Date I-Date I-Date O O O O O O B-Duration I-...


In [ ]:
print(f"Unique entities count:\nStrict: {len(unique_entities_strict)}\nLenient: {len(unique_entities_lenient)}")


42


In [ ]:
print(unique_entities_strict)

{'Area', 'Sex', 'Frequency', 'Dosage', 'Biological_attribute', 'Mass', 'Nonbiological_location', 'Severity', 'Other_event', 'Disease_disorder', 'Therapeutic_procedure', 'Texture', 'Family_history', 'Diagnostic_procedure', 'Qualitative_concept', 'Duration', 'Medication', 'Subject', 'Height', 'Outcome', 'Volume', 'Shape', 'Weight', 'Distance', 'Administration', 'Personal_background', 'Lab_value', 'Activity', 'Date', 'Sign_symptom', 'Quantitative_concept', 'Other_entity', 'Age', 'Clinical_event', 'O', 'History', 'Coreference', 'Color', 'Biological_structure', 'Occupation', 'Time', 'Detailed_description'}


In [ ]:
from collections import Counter

Counter(entities)

Counter({'O': 54914,
         'Age': 972,
         'Sex': 190,
         'Clinical_event': 826,
         'Biological_structure': 5537,
         'Sign_symptom': 4868,
         'Lab_value': 6555,
         'Diagnostic_procedure': 8730,
         'Detailed_description': 5756,
         'Disease_disorder': 2162,
         'Therapeutic_procedure': 1642,
         'Coreference': 437,
         'Date': 2135,
         'Duration': 779,
         'Texture': 81,
         'Medication': 1534,
         'Outcome': 51,
         'History': 1587,
         'Severity': 437,
         'Distance': 363,
         'Color': 96,
         'Frequency': 181,
         'Administration': 202,
         'Dosage': 1323,
         'Nonbiological_location': 717,
         'Time': 153,
         'Area': 183,
         'Personal_background': 70,
         'Activity': 150,
         'Quantitative_concept': 92,
         'Family_history': 474,
         'Qualitative_concept': 59,
         'Volume': 141,
         'Subject': 67,
         'Shape'